In [1]:
import yaml
import sys
import os 
import tensorflow as tf
sys.path.append('..')

import imp
import ml_pipeline
imp.reload(ml_pipeline)

<module 'ml_pipeline' from '../ml_pipeline/__init__.py'>

In [2]:
from ml_pipeline.audio import *
from ml_pipeline.sequence_hashing import *
from tensorflow.keras.models import *

In [3]:
c = yaml.load(open('../config/induction_config.yaml'))
print("Parameters: {}".format(c))
params       = WindowParams(c['spec_win'], c['spec_step'], c['fft_win'], c['fft_step'], c['highpass'])
inp          = c['input']
output       = c['output'] 
enc_path     = c['encoding']
enc          = load_model("{}/encoder.h5".format('../models/v2_lstm_v8/'))

/Users/daniel.kohlsdorf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """Entry point for launching an IPython kernel.


Parameters: {'spec_win': 128, 'spec_step': 64, 'fft_win': 512, 'fft_step': 64, 'highpass': 25, 'output': 'models/v2_lstm_v8/', 'input': 'gs://wdp-ds-data/dolphin_ogg', 'version': 'v2.0', 'encoding': 'data/encodings.csv'}


W0419 14:48:02.922008 4419687872 hdf5_format.py:177] No training configuration found in save file: the model was *not* compiled. Compile it manually.


In [4]:
sequences = []
for filename in tf.io.gfile.listdir('../data/unsupervised/'):
    if filename.endswith('.wav'):
        path = '../data/unsupervised/{}'.format(filename)
        x = [enc.predict(x.reshape(1, x.shape[0], x.shape[1], 1)).reshape((256)) for x, _, _, _  in spectrogram_windows(path, params)]
        if len(x) > 0:
            sequences.append(np.stack(x))
    if len(sequences) > 20:
        break

In [5]:
lbucket = length_bucketing(sequences, 10)
print(lbucket)

83
8
[ 4  0  5  0  0  0  2 10  0  1  2  0  2  3  1  5  0  1  0  0  1]


In [8]:
x = []
for sequence, l in zip(sequences, lbucket):
    if l == 2:
        x.append(sequence)
print(len(x), x[0].shape)
similarity_bucketing(x, 4, 2)

3 (16, 256)
(12, 256)
(12,)
(0, 0, 0, 0)
(1, 1, 1, 0)
(1, 0, 1, 1)


array([0, 1, 2], dtype=int32)